In [1]:
import pandas as pd
df_census = pd.read_csv('../data/MBTA_Website/census_data.csv')

df_census.columns = df_census.iloc[0]
df_census = df_census[1:]
df_census.reset_index(drop=True, inplace=True)

df_census.head()

,field concept,Total:,White alone,Black or African American alone,Hispanic or Latino,"Asian, Native Hawaiian and Pacific Islander alone, all ages","Other Races or Multiple Races, all ages",Total:,White alone,Black or African American alone,...,Nursing facilities/Skilled-nursing facilities,Other institutional facilities,Noninstitutionalized population:,College/University student housing,Military quarters,Other noninstitutional facilities,Total:,Occupied,Vacant,household size
0,Allston,24904,12536,1326,3259,6271,1512,23140,11976,1184,...,0,0,3281,3214,0,67,10748,10027,721,2.156477511
1,Back Bay,18190,13065,690,1208,2410,817,17042,12349,641,...,269,0,1610,1487,0,123,11524,10006,1518,1.630121927
2,Beacon Hill,9336,7521,252,537,630,396,8603,6980,231,...,0,0,33,0,0,33,6037,5485,552,1.696080219
3,Brighton,52047,32694,2414,5376,8703,2860,47657,30752,2076,...,266,56,3796,3493,0,303,23653,22535,1118,2.126292434
4,Charlestown,19120,13626,990,2075,1650,779,15661,11689,662,...,55,0,55,0,30,25,9525,8932,593,2.128302732


In [2]:
unique_columns = []

# Iterate through the existing column names
for column in df_census.columns:
    # If the column name is already in the list, append a suffix to make it unique
    while column in unique_columns:
        column = column + "_duplicate"
    unique_columns.append(column)

# Assign the unique column names to the DataFrame
df_census.columns = unique_columns


In [3]:
df_census.rename(columns={'field concept': 'Place'}, inplace=True)
df_census.columns

Index(['Place', 'Total:', 'White alone', 'Black or African American alone',
       'Hispanic or Latino',
       'Asian, Native Hawaiian and Pacific Islander alone, all ages',
       'Other Races or Multiple Races,  all ages', 'Total:_duplicate',
       'White alone_duplicate', 'Black or African American alone_duplicate',
       'Hispanic or Latino_duplicate',
       'Asian, Native Hawaiian and Pacific Islander alone, aged 18+',
       'Other Races or Multiple Races, aged 18+', 'Total: aged 0-17',
       'White alone, aged 0-17', 'Black or African American alone, aged 0-17',
       'Hispanic or Latino, aged 0-17',
       'Asian, Native Hawaiian and Pacific Islander alone, aged 0-17',
       'Other Races or Multiple Races, aged 0-17', 'household population',
       'Total:_duplicate_duplicate', 'Institutionalized population:',
       'Correctional facilities for adults', 'Juvenile facilities',
       'Nursing facilities/Skilled-nursing facilities',
       'Other institutional facilities'

In [4]:
df_census['Place'].unique().tolist()

['Allston',
 'Back Bay',
 'Beacon Hill',
 'Brighton',
 'Charlestown',
 'Chinatown',
 'Dorchester',
 'Downtown',
 'East Boston',
 'Fenway',
 'Hyde Park',
 'Jamaica Plain',
 'Longwood',
 'Mattapan',
 'Mission Hill',
 'North End',
 'Roslindale',
 'Roxbury',
 'South Boston',
 'South Boston Waterfront',
 'South End',
 'West End',
 'West Roxbury']

In [5]:
from geopy.geocoders import Nominatim

# List of place names
places = df_census['Place'].unique().tolist()

# Initialize a geocoder
geolocator = Nominatim(user_agent="get_coordinates")

# Create a dictionary to store place names, latitude, and longitude
coordinates_dict = {'Place': [], 'Latitude': [], 'Longitude': []}

# Iterate through the list of places and geocode each one
for place in places:
    location = geolocator.geocode(place)
    if location:
        coordinates_dict['Place'].append(place)
        coordinates_dict['Latitude'].append(location.latitude)
        coordinates_dict['Longitude'].append(location.longitude)

# Create a DataFrame from the coordinates
import pandas as pd

df_coordinates = pd.DataFrame(coordinates_dict)

# Print the DataFrame with place names, latitude, and longitude
print(df_coordinates)


                      Place   Latitude   Longitude
0                   Allston  42.355537  -71.132749
1                  Back Bay  42.350707  -71.079730
2               Beacon Hill  42.358708  -71.067829
3                  Brighton  50.821463   -0.140056
4               Charlestown  43.238348  -72.424713
5                 Chinatown  40.716491  -73.996250
6                Dorchester  50.713206   -2.437123
7                  Downtown  34.047996 -118.252100
8               East Boston  42.375097  -71.039217
9                    Fenway  42.345187  -71.104599
10                Hyde Park  51.507489   -0.162237
11            Jamaica Plain  42.311605  -71.114384
12                 Longwood  28.700850  -81.346558
13                 Mattapan  42.272321  -71.086995
14             Mission Hill  42.333265  -71.102029
15                North End  42.365097  -71.054495
16               Roslindale  42.291209  -71.124497
17                  Roxbury  41.556828  -73.308892
18             South Boston  42

In [6]:
df_census = df_coordinates.merge(df_census, on="Place")
df_census.head()

,Place,Latitude,Longitude,Total:,White alone,Black or African American alone,Hispanic or Latino,"Asian, Native Hawaiian and Pacific Islander alone, all ages","Other Races or Multiple Races, all ages",Total:_duplicate,...,Nursing facilities/Skilled-nursing facilities,Other institutional facilities,Noninstitutionalized population:,College/University student housing,Military quarters,Other noninstitutional facilities,Total:_duplicate_duplicate_duplicate,Occupied,Vacant,household size
0,Allston,42.355537,-71.132749,24904,12536,1326,3259,6271,1512,23140,...,0,0,3281,3214,0,67,10748,10027,721,2.156477511
1,Back Bay,42.350707,-71.079730,18190,13065,690,1208,2410,817,17042,...,269,0,1610,1487,0,123,11524,10006,1518,1.630121927
2,Beacon Hill,42.358708,-71.067829,9336,7521,252,537,630,396,8603,...,0,0,33,0,0,33,6037,5485,552,1.696080219
3,Brighton,50.821463,-0.140056,52047,32694,2414,5376,8703,2860,47657,...,266,56,3796,3493,0,303,23653,22535,1118,2.126292434
4,Charlestown,43.238348,-72.424713,19120,13626,990,2075,1650,779,15661,...,55,0,55,0,30,25,9525,8932,593,2.128302732


In [7]:
df_census.columns

Index(['Place', 'Latitude', 'Longitude', 'Total:', 'White alone',
       'Black or African American alone', 'Hispanic or Latino',
       'Asian, Native Hawaiian and Pacific Islander alone, all ages',
       'Other Races or Multiple Races,  all ages', 'Total:_duplicate',
       'White alone_duplicate', 'Black or African American alone_duplicate',
       'Hispanic or Latino_duplicate',
       'Asian, Native Hawaiian and Pacific Islander alone, aged 18+',
       'Other Races or Multiple Races, aged 18+', 'Total: aged 0-17',
       'White alone, aged 0-17', 'Black or African American alone, aged 0-17',
       'Hispanic or Latino, aged 0-17',
       'Asian, Native Hawaiian and Pacific Islander alone, aged 0-17',
       'Other Races or Multiple Races, aged 0-17', 'household population',
       'Total:_duplicate_duplicate', 'Institutionalized population:',
       'Correctional facilities for adults', 'Juvenile facilities',
       'Nursing facilities/Skilled-nursing facilities',
       'Other 

### Display Population Statistics

In [8]:
import folium

# Create a map centered around Boston
boston_map = folium.Map(location=[42.3601, -71.0589], zoom_start=12)

# Plot the place names using latitude and longitude
for i in range(len(df_coordinates)):
    name = df_census['Place'][i]
    lat = df_census['Latitude'][i]
    lon = df_census['Longitude'][i]

    # Get population statistics from the dataset
    white_population = int(df_census['White alone'][i])
    black_population = int(df_census['Black or African American alone'][i])
    hispanic_population = int(df_census['Hispanic or Latino'][i])
    asian_population = int(df_census['Asian, Native Hawaiian and Pacific Islander alone, all ages'][i])
    other_races = int(df_census['Other Races or Multiple Races,  all ages'][i]) 

    tot = white_population + black_population + hispanic_population + asian_population + other_races
    
    # Create an HTML popup with population statistics
    popup_html = f"<strong>{name}</strong><br>"
    # popup_html += f"Total Population: {total_population}<br>"
    popup_html += f"White: {round(white_population / tot, 4)}<br>"
    popup_html += f"Black: {round(black_population / tot, 4)}<br>"
    popup_html += f"Hispanic: {round(hispanic_population / tot, 4)}<br>"
    popup_html += f"Asian, Native Hawaiian, Pacific Islander: {round(asian_population / tot, 4)}<br>"
    popup_html += f"Other Races or Multiple Races: {round(other_races / tot, 4)}<br>"
    
    folium.Marker([lat, lon], tooltip=name, popup=folium.Popup(popup_html, max_width=400)).add_to(boston_map)

# Display the map
display(boston_map)

### Display Population Statistics against number of Stops

In [9]:
df_stops = pd.read_csv('../data/stops_locations.csv')
df_stops = df_stops.drop_duplicates(subset='stop_id')
print(df_stops.shape)
df_stops.head()

(5803, 5)


,object_id,stop_id,stop_name,X,Y
0,986,16535,Otis St @ Summer St,42.354243,-71.058557
1,1079,173,Saint James Ave @ Clarendon St,42.349935,-71.074649
2,1128,177,Saint James Ave @ Arlington St,42.351039,-71.070502
3,1138,178,Saint James Ave @ Dartmouth St,42.349505,-71.076639
4,1243,19031,400 Centre St - West,42.355938,-71.185299


In [10]:
import folium 
from folium.plugins import FastMarkerCluster
# Plot the bus stops using latitude and longitude
marker_cluster = FastMarkerCluster(data=list(zip(df_stops['X'], df_stops['Y'])))
marker_cluster.add_to(boston_map)
display(boston_map)